Fonction

In [ ]:
import plotly.express as px
from wordcloud import WordCloud
import nltk
import string
import numpy as np
from PIL import Image
import pandas as pd
from wordcloud import STOPWORDS
from nltk.corpus import stopwords


stopwords_custom = set(STOPWORDS)
stopwords_custom.update(["autre", "raison", "non", "don", "de", "la", "le", "et"])
WordCloud(..., stopwords=stopwords_custom)

def generate_wordcloud(column_data, colormap='Reds', mask_path=None):

    # Étape 1 : Nettoyer les données en supprimant les NaN
    cleaned_data = column_data.dropna().astype(str)
    
    # Étape 2 : Concaténer tout le texte en une seule chaîne
    texte = " ".join(cleaned_data)
    
    # Étape 3 : Nettoyer le texte (minuscules + suppression ponctuation)
    texte_nettoye = texte.lower().translate(str.maketrans('', '', string.punctuation))
    
    # Étape 4 : Filtrer les stopwords
    stop_words = set(stopwords.words('french'))
    mots = texte_nettoye.split()
    mots_filtres = [mot for mot in mots if mot not in stop_words]
    texte_filtre = ' '.join(mots_filtres)
    
    # Étape 5 : Charger le masque si fourni, sinon utiliser None
    mask = None
    if mask_path:
        mask = np.array(Image.open(mask_path))
        mask = 255 - mask  # Inverser le masque (fond noir, forme blanche)
    
    # Étape 6 : Générer le word cloud
    wordcloud = WordCloud(
        width=800,
        height=800,
        background_color='white',
        mask=mask,
        colormap=colormap
    ).generate(texte_filtre)
    
    # Étape 7 : Convertir en image et créer la figure Plotly
    image_wc = wordcloud.to_array()
    fig = px.imshow(image_wc)
    fig.update_layout(

        xaxis_visible=False,
        yaxis_visible=False,
        margin=dict(l=0, r=0, t=50, b=0)
    )
    
    return fig


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\STEEVE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Charger le DataFrame
df = pd.read_excel("last.xlsx")

Nuage de Mots pour les raisons de Temporarairement non eligible

In [3]:
# Tester avec une autre palette, par exemple 'Blues', sans masque
fig = generate_wordcloud(
    column_data = df[df['ÉLIGIBILITÉ_AU_DON.'] == 'Temporairement Non-eligible']['Raison_indisponibilité_fusionnée'].dropna(),
    colormap='Reds',
    mask_path='heart.png'
)



Nuage de Mots pour les raisons de temporairement  non eligible ; AUTRES RAISONS 


In [4]:
# Tester avec une autre palette, par exemple 'Blues', sans masque
fig = generate_wordcloud(
    column_data = df[df['ÉLIGIBILITÉ_AU_DON.'] == 'Temporairement Non-eligible']['Autre raisons,  preciser'].dropna(),
    colormap='Reds',
    mask_path='heart.png'
)

# Afficher le graphique
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [60]:
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
from wordcloud import WordCloud, STOPWORDS
import plotly.express as px
from collections import Counter
import re

def generate_blood_drop_wordcloud(series):

    # -------------------------------------------------------------
    # Sous-fonction : Créer un masque en forme de goutte de sang
    # -------------------------------------------------------------
    def create_drop_mask(width=800, height=800):
        img = Image.new('L', (width, height), 0)
        draw = ImageDraw.Draw(img)

        x0, y0 = width // 2, height // 3
        drop_width = width * 0.6
        drop_height = height * 0.7

        # Partie arrondie
        draw.ellipse([x0 - drop_width/2, y0, x0 + drop_width/2, y0 + drop_height], fill=255)

        # Triangle supérieur
        triangle_height = height * 0.2
        draw.polygon([
            (x0, y0 - triangle_height),
            (x0 - drop_width/4, y0),
            (x0 + drop_width/4, y0)
        ], fill=255)

        return np.array(img)

    # -------------------------------------------------------------
    # Sous-fonction : Nettoyage et comptage des mots
    # -------------------------------------------------------------
    def get_word_frequencies(series):
        # Nettoyage basique
        text = ' '.join(series.dropna().astype(str).str.lower())
        text = re.sub(r"[^a-zàâäéèêëîïôöùûüç\s\-]", " ", text)
        words = text.split()
        return Counter(words)

    # -------------------------------------------------------------
    # Étapes principales
    # -------------------------------------------------------------
    # Masque goutte
    mask = create_drop_mask()

    # Fréquences de mots
    frequencies = get_word_frequencies(series)

    # Stopwords personnalisés
    stopwords_custom = set(STOPWORDS)
    stopwords_custom.update(["autre", "raison", "non", "don", "de", "la", "le", "et", "l", "les", "des", "un", "une", "du", "en", "pour"])

    # Génération du nuage
    wordcloud = WordCloud(
        width=800,
        height=800,
        background_color='white',
        mask=255 - mask,
        colormap='Reds',
        stopwords=stopwords_custom
    ).generate_from_frequencies(frequencies)

    # Affichage avec Plotly
    image_wc = wordcloud.to_array()
    fig = px.imshow(image_wc)
    fig.update_layout(
    
        xaxis_visible=False,
        yaxis_visible=False,
        margin=dict(l=0, r=0, t=50, b=0)
    )
    fig.show()

    return wordcloud


Definitivement non eligible 

In [80]:
generate_blood_drop_wordcloud(df[df['ÉLIGIBILITÉ_AU_DON.'] == 'Définitivement non-eligible']['Raison_indisponibilité_fusionnée'].dropna())


Definitivement non eligible : autres raisons 

In [81]:
generate_blood_drop_wordcloud(df[df['ÉLIGIBILITÉ_AU_DON.'] == 'Définitivement non-eligible']['Si autres raison préciser'].dropna())

In [82]:
# Tester avec une autre palette, par exemple 'Blues', sans masque
fig = generate_wordcloud(
    column_data = df[df['ÉLIGIBILITÉ_AU_DON.'] == 'Définitivement non-eligible']['Si autres raison préciser'].dropna(),
    colormap='Reds',
    mask_path='heart.png'
)

# Afficher le graphique
fig.show()